## Create Operations Rehearsal Blocks from and Opsim data base

Take a OpsSim database created by the simulated scheduler for the 4th operations rehearsal and produce blocks for the for the scheduler CSC.

CWW - June 2024

In [1]:
from lsst.ts.observing import ObservingBlock, ObservingScript

from sqlalchemy import create_engine
# from astropy.io import fits
import pandas as pd

# import os
# from glob import glob
import matplotlib
import matplotlib.pyplot as pyplot

INFO:numexpr.utils:Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Let's get the data base into a Pandas DataFrame.  We will use sqlalchemy to make a database connection and then pandas to read the database tables into DataFrames.  This could have SQL selections in it but I am just reading the whole table in.

In [2]:
engine = create_engine('sqlite:////sdf/data/rubin/shared/ops-rehearsals/ops-rehearsal-4/scheduler_sims/ops_rehearsal_jun_2024_v4_playback_block.db')
Observations = pd.read_sql_table('observations', engine)
Observations.head()

,observationId,fieldRA,fieldDec,observationStartMJD,flush_by_mjd,visitExposureTime,filter,rotSkyPos,rotSkyPos_desired,numExposures,...,sunRA,sunDec,moonRA,moonDec,moonDistance,solarElong,moonPhase,cummTelAz,scripted_id,playbackBlock
0,0,225.082380,-39.955712,60474.949825,60474.995723,30.0,r,176.752550,0.0,1,...,82.437138,23.253969,171.647101,6.236051,67.406451,144.425887,48.233137,117.670196,0,0
1,1,225.156528,-40.001833,60474.950222,60474.995723,30.0,r,175.843583,0.0,1,...,82.437550,23.253989,171.649351,6.233816,67.475583,144.466782,48.234556,117.743282,0,0
2,2,225.153977,-39.996525,60474.950616,60474.995723,30.0,r,174.924306,0.0,1,...,82.437959,23.254009,171.651589,6.231592,67.468090,144.466152,48.235968,117.735238,0,0
3,3,225.117868,-40.137297,60474.951013,60474.995723,30.0,r,174.185915,0.0,1,...,82.438372,23.254028,171.653846,6.229349,67.511702,144.398682,48.237393,117.942886,0,0
4,4,224.814080,-40.054435,60474.951413,60474.995723,30.0,r,173.365103,0.0,1,...,82.438787,23.254048,171.656115,6.227095,67.266059,144.199598,48.238824,117.802497,0,0


In [3]:
# Summarize and then select columns we want to use

print(Observations.columns)

# Make a df with selected values.

visit_info = Observations[['observationId', 'fieldRA', 'fieldDec', 
    'observationStartMJD',
    'filter', 'rotSkyPos', 'rotTelPos', 
    'visitTime', 'visitExposureTime',
    'altitude', 'azimuth', 'slewTime',
    'note', 'target', 'fieldId',
    'block_id', 'playbackBlock']]

Index(['observationId', 'fieldRA', 'fieldDec', 'observationStartMJD',
       'flush_by_mjd', 'visitExposureTime', 'filter', 'rotSkyPos',
       'rotSkyPos_desired', 'numExposures', 'airmass', 'seeingFwhm500',
       'seeingFwhmEff', 'seeingFwhmGeom', 'skyBrightness', 'night', 'slewTime',
       'visitTime', 'slewDistance', 'fiveSigmaDepth', 'altitude', 'azimuth',
       'paraAngle', 'cloud', 'moonAlt', 'sunAlt', 'note', 'target', 'fieldId',
       'proposalId', 'block_id', 'observationStartLST', 'rotTelPos',
       'rotTelPos_backup', 'moonAz', 'sunAz', 'sunRA', 'sunDec', 'moonRA',
       'moonDec', 'moonDistance', 'solarElong', 'moonPhase', 'cummTelAz',
       'scripted_id', 'playbackBlock'],
      dtype='object')


In [4]:
#Print out the BLOCK-IDs.
pd.unique(visit_info.playbackBlock)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [5]:
#Group the data frame by block and print how many are in each block.
BLOCKS = visit_info.groupby(['playbackBlock'])
BLOCKS.observationId.count()

playbackBlock
0       60
1       60
2       60
3      120
4       60
      ... 
345     60
346     60
347     60
348    120
349      3
Name: observationId, Length: 350, dtype: int64

### Functions to extract the data frames we will use for the blocks.

In [6]:
%%script echo Skip this cell.  ROTPA fix is not needed for OR4.

# Build a dictionary of ROTPA keyed off of the observationID.  Go through all
# of the files in the pass3 output directory and get the ROTPA from the FITS
# headers.

image_directory = '/sdf/data/rubin/shared/ops-rehearsal-3/image_sims/pass_3'

rot_sky_dict = {}

with os.scandir(image_directory) as directory_iterator:
    for entry in directory_iterator:
        image_file_name = glob(entry.path+'/'+'*det000.fits.fz')[0]
        with fits.open(image_file_name) as fits_file:
            rot_sky_dict[int(entry.name)] = fits_file[0].header.get('ROTPA') 

Skip this cell. ROTPA fix is not needed for OR4.


In [7]:
def create_observation_data(selected_block):
    """
    Create observation data for a selected block.

    Parameters:
    - selected_block: The selected block for which observation data needs to be created.

    Returns:
    - obs_block: A data frame containing extracted data with correct field names.
    - visit_metadata: A data frame containing additional metadata information.
    """
    
    block = BLOCKS.get_group((selected_block,))

    # I need to also get the list of ROTPAs. I will use the observationIds as 
    # keys to the rot_sky_dict dictionary to produce a list of the angles.  I will
    # then replace the rotSkyPos column with this list.

    # Not needed
    # rot_pa = [rot_sky_dict[observation_id] for observation_id in block['observationId']]

    # Create two extracted data frames with the correct field names.  We will 
    # iterate over them later.  Also add some constant values to the data frame.
    # Both are for the take_image command, the second for the metadata information 
    # that will go into the parameters field.

    obs_block = block[['visitExposureTime', 'target', 'filter', 'slewTime']].copy()
    obs_block['filter'] = obs_block['filter'].replace(['i', 'r', 'g'], ['i_06', 'r_03', 'g_01'])
    obs_block.columns = ['exp_times', 'note', 'filter', 'slew_time']

    # Insert some extra static columns into the data frame
    obs_block.insert(len(obs_block.columns), 'image_type', 'OBJECT')
    obs_block.insert(len(obs_block.columns), 'nimages', 1)
    obs_block.insert(len(obs_block.columns), 'program', 'BLOCK-297')
    obs_block.insert(len(obs_block.columns), 'sim', True)

    # The field RA id in decimal degrees.  So we need to convert to decimal hours.
    # 
    # There is also a discrepancy between ROTPA and the rotSkyPos.
    # We will use the value of ROTPA we extracted from the image files instead.
    
    visit_metadata = block[['fieldRA', 'fieldDec', 'rotSkyPos']].copy()
    visit_metadata['fieldRA'] = visit_metadata['fieldRA'] / 15.0 
    # visit_metadata['rotSkyPos'] = rot_pa
    visit_metadata.columns = ['ra', 'dec', 'rot_sky']

    return obs_block, visit_metadata

### Functions to produce and write the scripts and blocks

In [8]:
# Function to print the json structure in a multi-line human readable 
# form for the blocks GitHub Repository.

def pretty_print_model(model):
    print(model.model_dump_json(indent=4))

In [9]:
# Function to actually produce an observing block from the data frames.
 
def create_observing_block(obs_block, visit_metadata):
    """
    Create an observing block based on the given observation block and visit metadata.

    Args:
        obs_block (pandas.DataFrame): The observation block containing the parameters for each row.
        visit_metadata (pandas.DataFrame): The visit metadata containing additional information for each row.

    Returns:
        tuple: A tuple containing the observing block (ObservingBlock object) and the number of exposures
        in the block.
    """

    # Set how many rows of the BLOCK to process
    rows = obs_block.shape[0]

    # All the scripts in the block will be appended to this list.
    script_list = []

    # First write the script that puts us into playlist mode.
    playlist_name = f'ops-rehearsal-4-{selected_block:03d}_playlist_{rows}'

    playlist_script = ObservingScript(name="run_command.py",  standard=True,
                parameters={"component": "CCCamera", "cmd": "play", 
                            "parameters": {"playlist": playlist_name, "repeat": False}})

    script_list.append(playlist_script)

    for row in range(rows):

        # Convert this row of the data frames parameters into dictionaries. These
        # lines convert the row of the data frame to a dictionary, then selects the
        # first element of the list of dictionaries. Also put a row counter into the
        # note field.
        obs_dict = obs_block.iloc[[row],:].to_dict('records')[0]
        obs_dict["note"] = f'{obs_dict["note"]} {selected_block:03d} {row + 1:02d}/{rows}'

        # We need to add the visit metadata to a separate dictionary and then add
        # it to the obs dictionary as a nested entry.
        metadata_dict = visit_metadata.iloc[[row],:].to_dict('records')[0]
        obs_dict["visit_metadata"] = metadata_dict

        # Place the the ComCam image parameters into a script
        take_image_script = ObservingScript(name="maintel/take_image_comcam.py", 
                                            standard=True, parameters = obs_dict)

        # Append it to the running script list    
        script_list.append(take_image_script)

    # Build the block dictionary
    observing_block = ObservingBlock(name=f'BLOCK-297', 
                                    program=f'ops-rehearsal-4-{selected_block:03d}', 
                                    scripts=script_list)
    return observing_block, rows


In [10]:
# Make an observing block and write it out.
# Observing block for operations rehearsal is 297.  See
# https://rubinobs.atlassian.net/browse/BLOCK-297
jira_observing_block = 297

# playback block 201 is the first of the three night of operations.
# playback block 251 is the last.
# playback_block_list = [208]
playback_block_list = range(201,252)

for selected_block in playback_block_list:

    # Create the observation data and metadata for the selected block
    obs_block, visit_metadata = create_observation_data(selected_block)

    # Print out the first five rows of the obs and metadata data frames
    print(f'Create observation data for playback block {selected_block}')

    # print("obs_block:")
    # display(obs_block.head())
    # print("Visit Metadata:")
    # display(visit_metadata.head())

    # Create the observing block
    print(f'Create observing block for playback block {selected_block}')
    observing_block, rows = create_observing_block(obs_block, visit_metadata)

    # Write out the observing block to a file. First determine directories and names.
    output_directory = '/sdf/home/c/cwalter/repos/ts_config_ocs/Scheduler/observing_blocks_ocs'
    sub_directory = f'BLOCK-{jira_observing_block:03d}'
    playlist_name = f'ops-rehearsal-4-{selected_block}_playlist_{rows}.json'
    file_name = f'{output_directory}/{sub_directory}/{playlist_name}'

    # Write out the observing block to screen
    # print("Observing Block:")
    # pretty_print_model(observing_block)

    with open(file_name, 'w') as file:
        print (f'Writing file {file_name}')
        file.write(observing_block.model_dump_json(indent=4))

Create observation data for playback block 201
Create observing block for playback block 201
Writing file /sdf/home/c/cwalter/repos/ts_config_ocs/Scheduler/observing_blocks_ocs/BLOCK-297/ops-rehearsal-4-201_playlist_60.json
Create observation data for playback block 202
Create observing block for playback block 202
Writing file /sdf/home/c/cwalter/repos/ts_config_ocs/Scheduler/observing_blocks_ocs/BLOCK-297/ops-rehearsal-4-202_playlist_60.json
Create observation data for playback block 203
Create observing block for playback block 203
Writing file /sdf/home/c/cwalter/repos/ts_config_ocs/Scheduler/observing_blocks_ocs/BLOCK-297/ops-rehearsal-4-203_playlist_120.json
Create observation data for playback block 204
Create observing block for playback block 204
Writing file /sdf/home/c/cwalter/repos/ts_config_ocs/Scheduler/observing_blocks_ocs/BLOCK-297/ops-rehearsal-4-204_playlist_60.json
Create observation data for playback block 205
Create observing block for playback block 205
Writing fi